# 오늘수업 : SCRAPING
### crawling과 scraping의 차이점
crawling은 웹페이지 전체를 다운로드 해야함, crawling 사이즈 큼, <br>
scarping은 특정한 데이터만 가져옴, 특정부분만 추출해서 사용<br>

# 전에 만들었던 다운로드 함수

In [2]:
from urllib import parse

header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

def getDownload(url, params={}, retries=3):
    resp = None
    
    try:
        resp = requests.get(url, params=params, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and retries > 0:
            print(retries)
            resp = getDownload(url, params, retries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
            
    return resp

In [1]:
import requests
from bs4 import BeautifulSoup

# 예제사이트로 scraping 하기

In [9]:
url = "http://example.webscraping.com/places/default/index"
html = getDownload(url)
dom = BeautifulSoup(html.content, 'lxml')

### 국가명 가져오기

In [15]:
nationNameList = [_.text.strip() for _ in dom.select("#results a")]

### 이미지 가져오기

In [25]:
imgList = [requests.compat.urljoin(url, _["src"]) for _ in dom.select("#results a > img")]

for src in imgList:
    img = getDownload(src)
    with open(src.split("/")[-1], "wb") as f:
        f.write(img.content)
#     print(src.split)
#     print(html.url)
    print(img.headers["Content-Type"].split('/')[0] == "image")     # content-type을 봐야함!!
#     break

True
True
True
True
True
True
True
True
True
True
